In [7]:
import pennylane as qml
import torch
from torch.utils.data import DataLoader, TensorDataset
import ray
from ray import tune
from ray.air import session
from sklearn.model_selection import train_test_split
import numpy as np
import yaml
import json
import time
import os
import pandas as pd


# Custom Libraries
from utils.model import Qkernel
from utils.data_generator import DataGenerator
from utils.agent import TrainModel

# Backend Configuration
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [8]:
# Read Configs
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

data = np.load('checkerboard_dataset.npy', allow_pickle=True).item()
x_train, x_test, y_train, y_test = data['x_train'], data['x_test'], data['y_train'], data['y_test']

training_data = torch.tensor(x_train, dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(x_test, dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(y_train, dtype=torch.int)
testing_labels = torch.tensor(y_test, dtype=torch.int)

kernel = Qkernel(   
                        device = config['qkernel']['device'], 
                        n_qubits = 4, 
                        trainable = True, 
                        input_scaling = True, 
                        data_reuploading = True, 
                        ansatz = 'embedding_paper', 
                        ansatz_layers = 5
                    )
    
agent = TrainModel(
                        kernel=kernel,
                        training_data=training_data,
                        training_labels=training_labels,
                        testing_data=testing_data,
                        testing_labels=testing_labels,
                        optimizer= 'adam',
                        lr= 0.1,
                        epochs = 200,
                        train_method= 'ccka',
                        target_accuracy=0.95,
                        get_alignment_every=10,  
                        validate_every_epoch=None, 
                        base_path='.',
                        lambda_kao=0.01,
                        lambda_co=0.1,
                        clusters=4
                      )

In [11]:
c1 = training_data[training_labels == 1]
c2 = training_data[training_labels == -1]

In [21]:
c1_cent = torch.mean(c1, dim = 0)
c2_cent = torch.mean(c2, dim = 0)

In [ ]:
c_cent = torch.tensor(torch.stack([c1_cent, c2_cent]).detach().numpy(), requires_grad=True).retain_grad()
c_cent

tensor([[0.4565, 0.4528],
        [0.4762, 0.5011]], requires_grad=True)

In [ ]:
ccs = [torch.tensor(torch.mean(cluster, dim=0).detach().numpy(), requires_grad=True) for cluster in torch.chunk(c1, 4)]
ccs = torch.tensor(torch.stack(ccs).detach().numpy(), requires_grad = True).retain_grad()

In [36]:
ccs

tensor([[0.2706, 0.4972],
        [0.5634, 0.5544],
        [0.4231, 0.3124],
        [0.6066, 0.4455]], requires_grad=True)

In [37]:
import torch.optim as optim
kernel_optimizer = optim.Adam([
            {'params': c_cent, 'lr': 0.1},
            {'params': ccs, 'lr': 0.1}
        ])

In [40]:
for param in ccs:
    if param.grad is not None:
        print(param.grad)

/tmp/ipykernel_9394/2082337650.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  if param.grad is not None:
